In [1]:
from sqlalchemy import create_engine
import pandas as pd


In [2]:
engine = create_engine(
    "mssql+pyodbc://AISHWARYARAMARA\SQLEXPRESS/ECOMMERCE_project"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)


In [3]:
pd.read_sql("SELECT TOP 5 * FROM purchase.customers_raw", engine)


C:\Users\DELL\anaconda3\Lib\site-packages\pandas\io\sql.py:1648: SAWarning: Unrecognized server version info '17.0.1000.7'.  Some SQL Server features may not function properly.
  con = self.exit_stack.enter_context(con.connect())


,customer_id,signup_date,segment
0,101,2021-01-10,regular
1,102,2020/03/15,Premium
2,103,None,REG
3,104,2019-07-01,None
4,101,2021-01-10,regular


In [5]:
df = pd.read_sql(
    "SELECT * FROM purchase.customers_raw",
    engine
)
df

,customer_id,signup_date,segment
0,101,2021-01-10,regular
1,102,2020/03/15,Premium
2,103,None,REG
3,104,2019-07-01,None
4,101,2021-01-10,regular
5,105,2022-11-20,premium
6,106,invalid_date,Regular


In [7]:
df['signup_date'] = pd.to_datetime(df['signup_date'],errors='coerce')

df['segment'] = df['segment'].str.upper().apply(lambda x : 'PREMIUM' if x == 'PREMIUM' else 'REGULAR')

In [13]:
df = df.sort_values(by='signup_date').drop_duplicates('customer_id',keep='last')


,customer_id,signup_date,segment
3,104,2019-07-01,REGULAR
4,101,2021-01-10,REGULAR
5,105,2022-11-20,PREMIUM
1,102,NaT,PREMIUM
2,103,NaT,REGULAR
6,106,NaT,REGULAR


In [14]:
df.to_sql(
    name='customer_clean_py',
    schema = 'purchase',
    con = engine,
    if_exists = 'replace',
    index = False
)

6

In [15]:
pd.read_sql(
    'SELECT * FROM purchase.customer_clean_py',
    engine
)

,customer_id,signup_date,segment
0,104,2019-07-01,REGULAR
1,101,2021-01-10,REGULAR
2,105,2022-11-20,PREMIUM
3,102,NaT,PREMIUM
4,103,NaT,REGULAR
5,106,NaT,REGULAR


In [17]:
print(df['customer_id'].is_unique)

True
